## 1. How Spark works

#### DAGScheduler (Directed Acyclic Graph scheduler)

- Shuffle은 join, groupBy같은 연산에서 일어남.
- 셔플을 기준으로 stage가 나뉨. 셔플이란 다른 파티션간의 데이터들의 합계 혹은 모종의 연산이 일어나는 경우.
- HDFS -> RDD -> Partition의 흐름으로 데이터가 읽힘.
- Partition : 실제 데이터를 논리적으로 나눠놓은 것. 데이터를 나누는 최소 단위라고도 볼 수 있음.
- 파티션된 데이터들이 repartition 되는 것이 셔플이라고도 볼 수 있음.
- 중요한 것은 하나의 파티션에 하나의 task가 돈다는 것.
- task(1 thread) : 1개의 task는 1개의 partition을 처리함.
- 하나의 stage 안에 있는 task를 묶은 것이 Taskset. 각각의 stage 안에 있는 task는 모두 같은 로직을 가지고 있음. 다른 로직을 가진 태스크는 없음.
- 이 모든 내용을 스케줄링해서 Worker에게 보내서 실행시킴.

-----

- (참고) spark 배포판이 아닌 오픈소스 라이브러리 모음. als도 원래 여기있다가 배포판으로 인정받음 : https://spark-packages.org
- (공홈) http://spark.apache.org/

## 2. Spark run

-----

#### Spark Start

In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SQLContext

val lines = sc.textFile("dataset/README.md")

lines = dataset/README.md MapPartitionsRDD[1] at textFile at <console>:31


dataset/README.md MapPartitionsRDD[1] at textFile at <console>:31

In [2]:
sc // docker에서는 4040 port를 열어줘야 접속이 가능. local에서는 4040 포트에서 UI monitoring을 제공.

Spark local-1536151443850: Some(http://520623a32092:4040)

In [4]:
spark

Spark local-1536151443850: Some(http://520623a32092:4040)

#### [LAB] RDD

In [1]:
val data = 1 to 1000

data = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170...


Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 22

###### 1부터 1000의 숫자는 master가 생성하지만, parallelize 하게 되면 1000개를 worker 노드들에 뿌린다.

In [2]:
val distData = sc.parallelize(data)

distData = ParallelCollectionRDD[0] at parallelize at <console>:29


ParallelCollectionRDD[0] at parallelize at <console>:29

In [4]:
distData.getNumPartitions // 1부터 500까지, 500부터 1000까지 데이터를 나눠서 파티션으로 관리하고 있음.

2

###### 여기서부터 쓰레드가 2개가 돌아가면서 working하는 것. (partition이 두개이기 때문에)

In [5]:
distData.filter(_ < 10).collect() // _ 는 tuple 형태로 만들겠다는 거고, collect는 action으로 뿌린걸 거둬들임.

[1, 2, 3, 4, 5, 6, 7, 8, 9]

<img src="images/lec_02-1.png">

-----
-----

#### [LAB] Shuffle by Repartition

In [9]:
val intRDD2 = sc.parallelize(1 to 20)

intRDD2 = ParallelCollectionRDD[3] at parallelize at <console>:27


ParallelCollectionRDD[3] at parallelize at <console>:27

In [10]:
intRDD2.saveAsTextFile("dataset/intRDD2") // intRDD2가 분산파일로 생성됨.

In [11]:
val intRDD5 = intRDD2.repartition(5) // 현재 분산파일로 저장된 파일의 개수는 2개였음 -> 파티션이 2개였는데 이걸 repartition 함.

intRDD5 = MapPartitionsRDD[8] at repartition at <console>:29


MapPartitionsRDD[8] at repartition at <console>:29

In [14]:
intRDD5.saveAsTextFile("dataset/intRDD5") // 파티션이 5개인 데이터로 5개 분산파일이 생성됨.

lastException: Throwable = null


###### repartition을 하면 shuffle이 됨. 그래서 2개짜리 파일에 있는 내용의 순서대로 저장되는 것이 아니라, 섞여서 저장됨.

<img src="images/lec_02-2.png">
-----

In [15]:
intRDD5.saveAsTextFile("dataset/intRDD55") // local에 저장되어있기 때문에 skip 됨.

-----
<img src="images/lec_02-3.png">

-----

-----
#### [LAB] Partitions for Files

- partition별 데이터가 unbalance 하다면 같은 stage 안에서 task 4개가 실행이 되어도, 특정 task의 partition 데이터 처리 하나가 안끝나서 해당 stage 속도가 느려질 수 있음. 즉 전체 성능 저하를 일으킴.

In [3]:
val rdd = sc.textFile("dataset/README.md")
rdd.getNumPartitions

rdd = dataset/README.md MapPartitionsRDD[5] at textFile at <console>:32


2

In [5]:
val rdd2 = sc.textFile("dataset/NOTICE")
rdd2.getNumPartitions

rdd2 = dataset/NOTICE MapPartitionsRDD[9] at textFile at <console>:32


2

In [9]:
val rdd3 = sc.textFile("dataset/README.md, dataset/NOTICE") // --> partition 3
val rdd4 = sc.textFile("LI*,NO*,RE*") // --> partition 4

lastException = null


Name: org.apache.hadoop.mapred.InvalidInputException
Message: Input path does not exist: file:/home/jovyan/work/ dataset/NOTICE
StackTrace:   at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
  at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
  at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:200)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
  at scala.Option.getOrElse(Option.scala:121)
  at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
  at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
  at scala.Option.getOrElse(Option.scala:121)
  at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
  at org.apache.spark.rdd.RDD.getNumPartitio

-----
<img src="images/lec_02-5.png">

-----

-----

#### [LAB] Cache
- Spark에서 Storage는 Cache를 의미함. UI에서 Storage는 cache된 데이터를 볼 수 있는 곳.
- cache도 부분적으로 lazy하게 실행되며, action이 있어야만 100%가 storage에 cache로써 저장됨.
- 컴퓨터의 메모리 용량에 따라, 메모리가 초과되면 오래된 cache 순으로 버려짐.

In [10]:
val data = sc.textFile("README.md") 
val distData = data.map(r => r + "_map")
distData.name = "distData" // 이름부여
distData.cache
distData.take(5)
// 웹 UI 확인 (Storage탭에서 Fraction Cached 부분이 100%가 아님을 확인)
// 웹 UI 확인 (Task도 1개만 실행되었음을 확인)

data = README.md MapPartitionsRDD[19] at textFile at <console>:30
distData = distData MapPartitionsRDD[20] at map at <console>:31


lastException: Throwable = null
distData.name: String = distData


[# Apache Spark_map, _map, Spark is a fast and general cluster computing system for Big Data. It provides_map, high-level APIs in Scala, Java, Python, and R, and an optimized engine that_map, supports general computation graphs for data analysis. It also supports a_map]

In [11]:
distData.collect
// 웹 UI 확인 (Storage탭에서 Fraction Cached 부분이 100%임을 확인)
// 웹 UI 확인 (Task도 모두 실행되었음을 확인)

[# Apache Spark_map, _map, Spark is a fast and general cluster computing system for Big Data. It provides_map, high-level APIs in Scala, Java, Python, and R, and an optimized engine that_map, supports general computation graphs for data analysis. It also supports a_map, rich set of higher-level tools including Spark SQL for SQL and DataFrames,_map, MLlib for machine learning, GraphX for graph processing,_map, and Spark Streaming for stream processing._map, _map, <http://spark.apache.org/>_map, _map, _map, ## Online Documentation_map, _map, You can find the latest Spark documentation, including a programming_map, guide, on the [project web page](http://spark.apache.org/documentation.html)._map, This README file only contains basic setup instructions._map, _map, ## Building Spark_map, _map, Spark is built using [Apache Maven](http://maven.apache.org/)._map, To build Spark and its example programs, run:_map, _map,     build/mvn -DskipTests clean package_map, _map, (You do not need to do t

In [12]:
distData.getStorageLevel // shell에서 캐시 확인

StorageLevel(memory, deserialized, 1 replicas)

In [13]:
distData.unpersist() // 캐시 삭제

distData MapPartitionsRDD[20] at map at <console>:31

-----
<img src="images/lec_02-5.png">

-----

-----

#### [LAB] Cache Size(Int vs String vs Object)
- Spark 에서 '23092'와 같은 string이 있다면, int type으로 바꿔서 cache, action등을 수행하는 것이 좋다.
- 같은 데이터라도 타입에 따라 cache의 size가 다르다.

In [14]:
val intRdd = sc.parallelize(1 to 10000) 
intRdd.name = "intRdd"
intRdd.cache
intRdd.count

intRdd = intRdd ParallelCollectionRDD[21] at parallelize at <console>:30


intRdd.name: String = intRdd


10000

In [15]:
val strRdd = intRdd.map(_.toString)
strRdd.name = "strRdd" 
strRdd.cache
strRdd.count

strRdd = strRdd MapPartitionsRDD[22] at map at <console>:32


strRdd.name: String = strRdd


10000

In [16]:
case class strCase (str: String)
val strCaseRdd = intRdd.map(x => strCase(x.toString)) 
strCaseRdd.name = "strCaseRdd" 
strCaseRdd.cache
strCaseRdd.count
// 웹 UI 확인 (Storage탭에서 캐시 Size 확인)

defined class strCase
strCaseRdd = strCaseRdd MapPartitionsRDD[23] at map at <console>:18


strCaseRdd.name: String = strCaseRdd


10000

-----

-----

#### [LAB] Get Cached RDD by Name
- 특정 캐시를 RDD이름으로 가져오거나 삭제 가능
- map(_._2) emits a second component of the tuple

In [17]:
val data = sc.textFile("README.md")
val distData = data.map(r => r + "_map")
data.setName("data")
distData.name = "distData"
data.cache
distData.cache
sc.getPersistentRDDs
distData.collect // action
// 웹 UI 확인 (distData만 action 실행했는데 data, distData 둘 다 캐시됨)

data = data MapPartitionsRDD[25] at textFile at <console>:33
distData = distData MapPartitionsRDD[26] at map at <console>:34


distData.name: String = distData


[# Apache Spark_map, _map, Spark is a fast and general cluster computing system for Big Data. It provides_map, high-level APIs in Scala, Java, Python, and R, and an optimized engine that_map, supports general computation graphs for data analysis. It also supports a_map, rich set of higher-level tools including Spark SQL for SQL and DataFrames,_map, MLlib for machine learning, GraphX for graph processing,_map, and Spark Streaming for stream processing._map, _map, <http://spark.apache.org/>_map, _map, _map, ## Online Documentation_map, _map, You can find the latest Spark documentation, including a programming_map, guide, on the [project web page](http://spark.apache.org/documentation.html)._map, This README file only contains basic setup instructions._map, _map, ## Building Spark_map, _map, Spark is built using [Apache Maven](http://maven.apache.org/)._map, To build Spark and its example programs, run:_map, _map,     build/mvn -DskipTests clean package_map, _map, (You do not need to do t

In [18]:
sc.getPersistentRDDs

Map(25 -> data MapPartitionsRDD[25] at textFile at <console>:33, 21 -> intRdd ParallelCollectionRDD[21] at parallelize at <console>:30, 22 -> strRdd MapPartitionsRDD[22] at map at <console>:32, 26 -> distData MapPartitionsRDD[26] at map at <console>:34, 23 -> strCaseRdd MapPartitionsRDD[23] at map at <console>:18)

In [19]:
sc.getPersistentRDDs.filter(x => x._2.name.equals("data")).foreach(x => x._2.unpersist()) // 특정 캐시("data")만 제거
sc.getPersistentRDDs.foreach(x => x._2.unpersist()) // 모든 캐시 제거
distData.collect // action
// 웹 UI 확인 (캐시 없음. 기존 캐시 모두 unpersist를 통해 설정 해제 됨)

[# Apache Spark_map, _map, Spark is a fast and general cluster computing system for Big Data. It provides_map, high-level APIs in Scala, Java, Python, and R, and an optimized engine that_map, supports general computation graphs for data analysis. It also supports a_map, rich set of higher-level tools including Spark SQL for SQL and DataFrames,_map, MLlib for machine learning, GraphX for graph processing,_map, and Spark Streaming for stream processing._map, _map, <http://spark.apache.org/>_map, _map, _map, ## Online Documentation_map, _map, You can find the latest Spark documentation, including a programming_map, guide, on the [project web page](http://spark.apache.org/documentation.html)._map, This README file only contains basic setup instructions._map, _map, ## Building Spark_map, _map, Spark is built using [Apache Maven](http://maven.apache.org/)._map, To build Spark and its example programs, run:_map, _map,     build/mvn -DskipTests clean package_map, _map, (You do not need to do t